# GoogleDriver

In [4]:
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os
import random

In [5]:
def item_info_part1(div):
    href = list(div.children)[0]['href']
    title = list(list(div.children)[0].children)[0].text
    return href, title


def item_full_info(tile):
    res = []
    try:
        div = tile.find('div', class_="iva-item-title-py3i_")
        res.extend(item_info_part1(div))
    except AttributeError:
        res.extend([pd.NA, pd.NA])
    try:
        div = tile.find('strong', class_="styles-module-root-bLKnd")
        res.append(next(div.childGenerator()).text)
    except AttributeError:
        res.append(pd.NA)
    try:
        div = next(tile.find('div', class_="iva-item-userInfoStep-dWwGU").find('div', class_ = 'style-root-uufhX').childGenerator())
        res.append(div['href']) 
    except AttributeError:
        res.append(pd.NA)
    try:
        res.extend([div.find('p').text, div.find('span', {'data-marker':"seller-rating/score"}).text])
    except AttributeError:
        res.extend([pd.NA,pd.NA])
    try:
        res.append(next(tile.find('div', class_='iva-item-descriptionStep-C0ty1').childGenerator()).text)
    except AttributeError:
        res.append(pd.NA)
    try:
        res.append(tile.find('div', class_='iva-item-dateInfoStep-_acjp').find('p', {'data-marker':'item-date'}).text)
    except AttributeError:
        res.append(pd.NA)

    

    return res

In [6]:
def get_screenshot(driver,core, item, cnt, q_str):
    driver.get(core+item)
    driver.implicitly_wait(random.randint(1,4))
    driver.get_screenshot_as_file(f'{q_str}/{cnt}.png')

In [8]:
# print(ChromeDriverManager().install())
executable_path = '/Users/ilyapchelintsev/.wdm/drivers/chromedriver/mac64/129.0.6668.100/chromedriver-mac-arm64/chromedriver'
service = Service(executable_path=executable_path)
driver = webdriver.Chrome(service=service)
q_str = 'Газпромбанк установка'

#os.mkdir(q_str)
core = 'https://www.avito.ru'
main = '/all/predlozheniya_uslug'
param_head = '?'
query = 'q=' + '+'.join(q_str.split())
page_n = lambda x: f'p={x}&' if x > 1 else ''
data = []
i = 1
cnt = 0
wait = WebDriverWait(driver, 50)
while True:
    url = core + main + param_head + page_n(i) + query.replace(' ', '+')
    driver.get(url)
    wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'iva-item-content-rejJg')))
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    if i == 1:
        qnt_query = soup.find('span', class_='page-title-count-wQ7pG')
        qnt = 0 if qnt_query is None else int(qnt_query.text) 
        print(f'Получено {qnt} записей, обработка...')
        if qnt == 0:
            break
    no_new_data = True
    tiles = soup.find_all('div', class_ = 'iva-item-content-rejJg')
    for t in tiles:
        res = item_full_info(t)
        #if res[0] is not pd.NA:
        #    get_screenshot(driver, core, res[0], cnt, q_str)
        data.append(res)
        cnt += 1
        no_new_data = False
    print(f'Страница {i} загружена, {cnt} записей загружено')
    if cnt == qnt or no_new_data:
        break
    else:
        i += 1

Получено 536 записей, обработка...
Страница 1 загружена, 50 записей загружено
Страница 2 загружена, 100 записей загружено
Страница 3 загружена, 150 записей загружено
Страница 4 загружена, 200 записей загружено
Страница 5 загружена, 250 записей загружено
Страница 6 загружена, 300 записей загружено
Страница 7 загружена, 336 записей загружено


KeyboardInterrupt: 

In [14]:
driver.quit()
df = pd.DataFrame(data, columns=['Ссылка', 'Название', 'Цена', 'Компания (ссылка)', 'Компания',
       'Рейтинг', 'Описание', 'Дата создания'])

In [30]:
driver.quit()

In [10]:
print('Not full:',(df.isna().sum(axis=1) > 0).sum().item(),'of', df.shape[0])
print('Empty:',(df.isna().sum(axis=1) == df.shape[0]).sum().item(),'of', df.shape[0])

Not full: 107 of 336
Empty: 0 of 336


In [41]:
dfe = Parse('Абырвалг')

Получено 0 записей, обработка...


In [11]:
df.head(20)

,Ссылка,Название,Цена,Компания (ссылка),Компания,Рейтинг,Описание,Дата создания
0,/balakovo/predlozheniya_uslug/ustanovka_udalen...,Установка удаленных приложений на iPhone,1 000 ₽,/brands/jobscalling_balakovo?src=search_seller...,JOBSCALLING-МАГАЗИН ТЕХНИКИ APPLE и АКСЕССУАРО...,"5,0",Установим приложение вконтакте (Vk) Сбербанк 2...,1 неделю назад
1,/ufa/predlozheniya_uslug/ustanovka_sberbank_ti...,Установка Сбербанк Tinkoff Газпромбанк,100 ₽ за услугу,/brands/0d0d2b8d6a81a9efb6df605658a11159?src=s...,Ufa,"4,7",1. Цена ( Наилучшая цена на рынке).\n2. Быстро...,6 дней назад
2,/omsk/predlozheniya_uslug/ustanovka_prilozheni...,Установка приложений на iPhone,от 140 ₽,/brands/c25aac526cefcaa5089940c6b95b215e?src=s...,Виктор Бахтин,"5,0",Я установлю на ваше устройство такие приложени...,16 сентября 21:35
3,/moskva/predlozheniya_uslug/ustanovka_prilozhe...,Установка приложений Сбербанка и др. на Айфон,200 ₽ за услугу,/brands/i28899352?src=search_seller_info,Владимир,"5,0",Установлю сбербанк и другие приложения на Айфо...,5 дней назад
4,/izhevsk/predlozheniya_uslug/ustanovka_sberban...,"Установка сбербанка, альфа банка, втб на айфон",от 700 ₽ за услугу,<NA>,<NA>,<NA>,Только для жителей города Ижевск!\n\nУстановлю...,16 сентября 18:13
5,/staryy_oskol/predlozheniya_uslug/ustanovka_sb...,Установка Сбербанк.Тинькофф. втб. псб.Газпромбанк,250 ₽ за услугу,<NA>,<NA>,<NA>,Установка на iPhone банков. Открытие. Альфа-Ба...,2 недели назад
6,/sankt-peterburg/predlozheniya_uslug/ustanovka...,Установка Сбер/ржд/DJI/Тиньк/ситидрайв,Бесплатно,/brands/i49574512?src=search_seller_info,RemLabs - Савушкина 11,"4,9",Восстанавливаем удалённые из appstore приложен...,1 неделю назад
7,/noyabrsk/predlozheniya_uslug/ustanovka_priloz...,Установка приложений на iPhone,400 ₽ за услугу,<NA>,<NA>,<NA>,Добрый день!\nУстановлю на ваш iPhone приложен...,3 недели назад
8,/moskva/predlozheniya_uslug/ustanovka_na_iphon...,Установка на iPhone банковких приложений,400 ₽ за услугу,/brands/3d693b6f0e5b5d1a217f91a536ca8c9d?src=s...,Эрдман,"5,0",В наличии следующие банки:\nСбербанк.\nАльфа-б...,4 дня назад
9,/moskva/predlozheniya_uslug/ustanovka_sberbank...,"Установка сбербанка, сбербизнеса, мэш и тд",200 ₽,/brands/d0faba648425f8bbe77f772fd0df263a?src=s...,Коллекционер-ру,"4,8",Здравствуйте! Онлайн и офлайн установка любого...,1 неделю назад


In [13]:
df.iloc[250:260]

,Ссылка,Название,Цена,Компания (ссылка),Компания,Рейтинг,Описание,Дата создания
250,/novosibirsk/predlozheniya_uslug/nastroyka_ets...,"Настройка эцп и торговых площадок еис, аст-гоз",от 450 ₽ за услугу,/brands/i68352066?src=search_seller_info,Install-ECP,"5,0",Добрый день! Мое имя Александр.\n\nУстановлю Э...,3 недели назад
251,/tomsk/predlozheniya_uslug/nastroyka_etsp_i_to...,"Настройка эцп и торговых площадок еис, аст-гоз",от 500 ₽ за услугу,/brands/i68352066?src=search_seller_info,Install-ECP,"5,0",Добрый день! Мое имя Александр!\n\nЕсли у вас ...,3 недели назад
252,/krasnodar/predlozheniya_uslug/agent_na_torgah...,Агент на торгах. Конфискат Банкротство Должники,от 1 000 ₽ за услугу,/brands/i175345009?src=search_seller_info,Сергей,"5,0",Возник вопрос покупки через торги? Под ключ?\n...,2 недели назад
253,/belgorod/predlozheniya_uslug/agent_na_torgah....,Агент на торгах. Конфискат Банкротство Должники,от 1 000 ₽ за услугу,/brands/i175345009?src=search_seller_info,Сергей,"5,0",Возник вопрос покупки через торги? Под ключ?\n...,2 недели назад
254,/krasnoyarsk/predlozheniya_uslug/agent_na_torg...,Агент на торгах. Конфискат Банкротство Должники,от 1 000 ₽ за услугу,/brands/i175345009?src=search_seller_info,Сергей,"5,0",Возник вопрос покупки через торги? Под ключ?\n...,2 недели назад
255,/domodedovo/predlozheniya_uslug/karkasnyy_dom_...,Каркасный дом в ипотеку / Строительство под ключ,52 500 ₽,/brands/f9285d1747293c82150e93687275c361?src=s...,<NA>,<NA>,Ваш идеальный дом начинается здесь: строительс...,1 неделю назад
256,/kirovskaya_oblast_kirov/predlozheniya_uslug/a...,Агент на торгах. Конфискат Банкротство Должники,от 1 000 ₽ за услугу,/brands/i175345009?src=search_seller_info,Сергей,"5,0",Возник вопрос покупки через торги? Под ключ?\n...,5 дней назад
257,/novosibirsk/predlozheniya_uslug/agent_na_torg...,Агент на торгах. Конфискат Банкротство Должники,от 1 000 ₽ за услугу,/brands/i175345009?src=search_seller_info,Сергей,"5,0",Возник вопрос покупки через торги? Под ключ?\n...,2 недели назад
258,/lyubertsy/predlozheniya_uslug/stroitelstvo_ka...,"Строительство каркасных домов, коттеджей, дач",52 500 ₽,/brands/f9285d1747293c82150e93687275c361?src=s...,<NA>,<NA>,Ваш идеальный дом начинается здесь: строительс...,2 недели назад
259,/petrozavodsk/predlozheniya_uslug/agent_na_tor...,Агент на торгах. Конфискат Банкротство Должники,от 1 000 ₽ за услугу,/brands/i175345009?src=search_seller_info,Сергей,"5,0",Возник вопрос покупки через торги? Под ключ?\n...,5 дней назад
